## 中文自然语言处理分析
by 寒小阳(hanxiaoyang.ml@gmail.com)

和拉丁语系不同，亚洲语言是不用空格分开每个有意义的词的。而当我们进行自然语言处理的时候，大部分情况下，词汇是我们对句子和文章理解的基础，因此需要一个工具去把完整的文本中分解成粒度更细的词。

### 1、关键词提取
#### 基于 TF-IDF 算法的关键词抽取

import jieba.analyse

- jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())

 - sentence 为待提取的文本

 - topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20

 - withWeight 为是否一并返回关键词权重值，默认值为 False

 - allowPOS 仅包括指定词性的词，默认值为空，即不筛选

In [1]:
import jieba.analyse as analyse
import pandas as pd
df = pd.read_csv("/usr/local/codeData/jupyterData/jupyter-bd/textCategorization/data/technology_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()
content = "".join(lines)
print ("  ".join(analyse.extract_tags(content, topK=30, withWeight=False, allowPOS=())))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.666 seconds.
Prefix dict has been built successfully.


用户  2016  互联网  手机  平台  人工智能  百度  2017  智能  技术  数据  360  服务  直播  产品  企业  安全  视频  移动  应用  网络  行业  游戏  机器人  电商  内容  中国  领域  通过  发展


In [2]:
import jieba.analyse as analyse
import pandas as pd
df = pd.read_csv("/usr/local/codeData/jupyterData/jupyter-bd/textCategorization/data/military_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()
content = "".join(lines)
print ("  ".join(analyse.extract_tags(content, topK=30, withWeight=False, allowPOS=())))

航母  训练  海军  中国  官兵  部队  编队  作战  10  任务  美国  导弹  能力  20  2016  军事  无人机  装备  进行  记者  我们  军队  安全  保障  12  战略  军人  日本  南海  战机


#### 基于 TextRank 算法的关键词抽取
- jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v')) 直接使用，接口相同，注意默认过滤词性。
- jieba.analyse.TextRank() 新建自定义 TextRank 实例

算法论文： TextRank: Bringing Order into Texts

基本思想:

- 将待抽取关键词的文本进行分词
- 以固定窗口大小(默认为5，通过span属性调整)，词之间的共现关系，构建图
- 计算图中节点的PageRank，注意是无向带权图

In [3]:
import jieba.analyse as analyse
import pandas as pd
df = pd.read_csv("/usr/local/codeData/jupyterData/jupyter-bd/textCategorization/data/military_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()
content = "".join(lines)

print ("  ".join(analyse.textrank(content, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))))
print ("---------------------我是分割线----------------")
print ("  ".join(analyse.textrank(content, topK=20, withWeight=False, allowPOS=('ns', 'n'))))

中国  海军  训练  美国  部队  进行  官兵  航母  作战  任务  能力  军事  发展  工作  国家  问题  建设  导弹  编队  记者
---------------------我是分割线----------------
中国  海军  美国  部队  官兵  航母  军事  国家  任务  能力  导弹  技术  问题  日本  军队  编队  装备  系统  记者  战略


### 2、LDA主题模型
咱们来用LDA主题模型建模，看看这些新闻主要在说哪些topic。

首先我们要把文本内容处理成固定的格式，一个包含句子的list，list中每个元素是分词后的词list。类似下面这个样子。

[[第，一，条，新闻，在，这里],[第，二，条，新闻，在，这里],[这，是，在，做， 什么],...]

In [4]:
from gensim import corpora, models, similarities
import gensim

#### 载入停用词

In [5]:
stopwords=pd.read_csv("/usr/local/codeData/jupyterData/jupyter-bd/textCategorization/data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

In [6]:
stopwords

array(['!', '"', '#', ..., '07', '08', '09'], dtype=object)

#### 转换成合适的格式

In [7]:
import jieba
import pandas as pd
df = pd.read_csv("/usr/local/codeData/jupyterData/jupyter-bd/textCategorization/data/technology_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()

sentences=[]
for line in lines:
    try:
        segs=jieba.lcut(line)
        segs = list(filter(lambda x:len(x)>1 and x not in stopwords , segs))
#         segs = filter(lambda x:len(x)>1, segs)
#         segs = filter(lambda x:x not in stopwords, segs)
        sentences.append(segs)
    except Exception:
        print (line)
        continue

In [8]:
for word in sentences[5]:
    print (word)

本次
商汤
带来
黄仁勋
展示
遥相呼应
SenseFace
人脸
布控
系统
千万级
人员
库中
300ms
识别
瞬间
锁定目标
功耗
十几
当属
人脸
布控
一大
科技


#### 词袋模型

In [9]:
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]

### LDA建模

In [10]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

我们查一下第3号分类，其中最常出现的单词是：

In [11]:
print (lda.print_topic(3, topn=5))

0.038*"中国" + 0.031*"科技" + 0.018*"互联网" + 0.015*"创新" + 0.014*"产业"


我们把所有的主题打印出来看看

In [12]:
for topic in lda.print_topics(num_topics=20, num_words=8):
    print (topic[1])

0.034*"直播" + 0.030*"公司" + 0.018*"游戏" + 0.016*"汽车" + 0.016*"业务" + 0.014*"亿元" + 0.013*"投资" + 0.012*"增长"
0.050*"内容" + 0.033*"视频" + 0.020*"平台" + 0.012*"生态" + 0.011*"增长" + 0.010*"创业" + 0.009*"OTT" + 0.008*"媒体"
0.028*"孩子" + 0.027*"共享" + 0.022*"家长" + 0.019*"单车" + 0.015*"无人机" + 0.013*"手机" + 0.012*"出行" + 0.011*"小时"
0.038*"中国" + 0.031*"科技" + 0.018*"互联网" + 0.015*"创新" + 0.014*"产业" + 0.010*"2017" + 0.010*"企业" + 0.009*"发展"
0.030*"技术" + 0.025*"发展" + 0.021*"人工智能" + 0.015*"企业" + 0.015*"领域" + 0.014*"互联网" + 0.013*"行业" + 0.013*"数据"
0.019*"网秦" + 0.016*"知识" + 0.016*"付费" + 0.013*"费用" + 0.012*"陌陌" + 0.012*"视频" + 0.011*"社交" + 0.009*"平台"
0.031*"旅游" + 0.030*"酒店" + 0.022*"携程" + 0.012*"服务" + 0.012*"外卖" + 0.011*"旅行" + 0.010*"AMD" + 0.010*"网络攻击"
0.040*"百度" + 0.035*"用户" + 0.016*"手机" + 0.015*"地图" + 0.013*"搜索" + 0.013*"功能" + 0.011*"360" + 0.009*"APP"
0.027*"信息" + 0.024*"用户" + 0.018*"网络" + 0.017*"支付" + 0.017*"诈骗" + 0.013*"软件" + 0.012*"手机" + 0.009*"保护"
0.032*"学习" + 0.021*"驾驶" + 0.020*"机器" + 0.018*"算法" + 0.018*"百度" + 0.01